In [1]:
import time
import datetime
import mysql.connector
###########################
import os
import cv2
import math
import copy
import numpy as np
import skimage.io as io
from scipy.ndimage import find_objects
from skimage.feature import canny
import matplotlib.pyplot as plt
from skimage.color import rgb2gray , rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.filters import threshold_otsu, threshold_mean, threshold_li, threshold_isodata, threshold_niblack , median
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin , disk
from skimage.draw import rectangle, rectangle_perimeter
from skimage.transform import resize
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def multi_color_space_model_es24(rgba_frame , hsv_frame , ycbcr_frame):
    B_es24 = rgba_frame[:,:,0]
    G_es24 = rgba_frame[:,:,1]
    R_es24 = rgba_frame[:,:,2]
    A_es24 = rgba_frame[:,:,3]
    H_es24 = hsv_frame[:,:,0]
    S_es24 = hsv_frame[:,:,1]
    V_es24 = hsv_frame[:,:,2]
    Y_es24 = ycbcr_frame[:,:,0]
    Cr_es24 = ycbcr_frame[:,:,1]
    Cb_es24 = ycbcr_frame[:,:,2]
    n_r_es24 = np.ma.divide(R_es24, G_es24)
    n_b_es24 = np.ma.divide(B_es24, G_es24)
    log_rg_es24 = np.ma.log( n_r_es24 )
    log_bg_es24 = np.ma.log( n_b_es24 )
    condition_1_es24 = (log_rg_es24>=0.15) & (log_rg_es24<=1.1)
    condition_2_es24 = (log_bg_es24>=-4) & (log_bg_es24<=0.3)
    mask_1_es24 = condition_1_es24 & condition_2_es24
    mask_rgb_es24 = (R_es24>95)&(G_es24>40)&(B_es24>20)&(R_es24>G_es24)&(R_es24>B_es24)&(abs(R_es24-G_es24)>15)&(A_es24>15)
    mask_Ycbcr_es24 = (Cr_es24 > 135)&(Cb_es24>85)&(Y_es24>80)&(Cr_es24 <= (1.5862*Cb_es24)+20)&(Cr_es24>=(0.3448*Cb_es24)+76.2069)&(Cr_es24 >= (-4.5652*Cb_es24)+234.5652)&(
                  Cr_es24 <= (-1.15*Cb_es24)+301.75)&(Cr_es24 <= (-2.2857*Cb_es24)+432.85)
    mask_hsv_es24 = ((0.0 <= H_es24) & (H_es24 <= 50.0))&((0.23 <= S_es24/255) & (S_es24/255 <= 0.68))
    mask1_es24 = mask_rgb_es24 & mask_Ycbcr_es24
    mask2_es24 = mask_rgb_es24 & mask_hsv_es24
    return mask_1_es24

In [3]:
def dist_es24(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [4]:
def moving_area(currentFrameGray , lastFrameGray ,beforeLastFrameGray , threshold):
    diff1 = cv2.absdiff(currentFrameGray,lastFrameGray)
    diff2 = cv2.absdiff(currentFrameGray,beforeLastFrameGray)    
    _, binary1 = cv2.threshold(diff1,threshold,1,cv2.THRESH_BINARY)
    _, binary2 = cv2.threshold(diff2,threshold,1,cv2.THRESH_BINARY)
    resultImageDiff = cv2.bitwise_and(binary1, binary2)
    resultImageDiff = cv2.erode(resultImageDiff, np.ones((5, 5),dtype='uint8'), iterations = 2)
    objs = find_objects(resultImageDiff)
    if not objs:
        mROI=[0,0,0,0]
    else:        
        mROI=[objs[0][0].start,objs[0][0].stop,objs[0][1].start,objs[0][1].stop]
    moving_region = np.zeros(currentFrameGray.shape)
    moving_region[mROI[0]:mROI[1] , mROI[2]:mROI[3]] = 1
    return moving_region

In [5]:
def findBoundingBox_es24(cnt_es24):
    Xmax_es24 = max(cnt_es24[:,0,0])
    Ymax_es24 = max(cnt_es24[:,0,1])
    Xmin_es24 = min(cnt_es24[:,0,0])
    Ymin_es24 = min(cnt_es24[:,0,1])
    box_es24 = [Xmin_es24, Ymin_es24, Xmax_es24, Ymax_es24]
    boxWidth_es24 = Xmax_es24 - Xmin_es24
    boxHeight_es24 = Ymax_es24 - Ymin_es24
    boxAspectRatio_es24 = boxHeight_es24 / boxWidth_es24
    return box_es24, boxWidth_es24, boxHeight_es24, boxAspectRatio_es24

In [6]:
def Left_index_es24(points_es24): 
    minn_es24 = 0
    for i in range(1,len(points_es24)): 
        if points_es24[i,0,0] < points_es24[minn_es24,0,0]: 
            minn_es24 = i 
        elif points_es24[i,0,0] == points_es24[minn_es24,0,0]: 
            if points_es24[i,0,1] > points_es24[minn_es24,0,1]: 
                minn_es24 = i 
    return minn_es24 

In [7]:
def findOrientation_es24(p, q, r): 
    val_es24 = (q[0,1] - p[0,1]) * (r[0,0] - q[0,0]) - (q[0,0] - p[0,0]) * (r[0,1] - q[0,1]) 
    if val_es24 == 0: 
        return 0
    elif val_es24 > 0: 
        return 1 
    return 2

In [8]:
def convexHull_es24(data, returnPoints = True): 
    points_es24 = copy.deepcopy(data)
    n = len(points_es24)
    if n < 3: 
        return
    l =  Left_index_es24(points_es24)
    hull = [] 
    p = l 
    q = 0
    while(True):    
        hull.append(p) 
        q = (p + 1) % n 
        for i in range(n): 
            if(findOrientation_es24(points_es24[p],points_es24[i], points_es24[q]) == 2): 
                q = i   
        p = q 
        if(p == l): 
            break
    if returnPoints:
        return points_es24[hull,:,:] 
    old_shape = len(hull)
    hull = np.reshape(hull,(old_shape,1))
    return hull

In [9]:
def getVMove_es24 (hull, vAnchor):
    vLines = []
    vPoints = []
    lastPoint = hull[0,:,:]
    for i in hull:
        if(dist_es24(i[0,:], vAnchor) > 80 and np.absolute(i[0,0] - lastPoint[0,0]) > 10):
            vPoints.append((i[0, 0], i[0, 1]))
            vLines.append([(int(i[0,0]),int(i[0,1])), (int(vAnchor[0]), int(vAnchor[1]))])
            lastPoint = i
    return vLines, vPoints

In [10]:
def getHMove_es24(hull, hAnchor):
    hLines = []
    hPoints = []
    lastPoint = hull[0,:,:]
    for i in hull:
        if(dist_es24(i[0,:],hAnchor) > 80 and dist_es24(i[0,:], lastPoint[0,:]) > 10):        
            hPoints.append((i[0, 0], i[0, 1]))
            hLines.append([(int(i[0,0]),int(i[0,1])), (int(hAnchor[0]), int(hAnchor[1]))])
            lastPoint = i
    return hLines, hPoints

In [11]:
def getMoveNo(cnt_es24, frame):
    hull = cv2.convexHull(cnt_es24, returnPoints=False)
    defects = cv2.convexityDefects(cnt_es24, hull)
    upMove = 0
    downMove = 0
    leftMove = 0
    leftMoveRight = 0
    rightMoveAcute = 0
    rightMoveRight = 0
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt_es24[s][0])
        end = tuple(cnt_es24[e][0])
        far = tuple(cnt_es24[f][0])
        pt= (100,180)
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        s = (a+b+c)/2
        ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
        d=(2*ar)/a
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if (far[1] > start[1]) & (far[1] > end[1]) & (far[0] < start[0]) & (far[0] > end[0]): 
            if (angle <= 70 and d > 30):
                upMove += 1
                cv2.circle(frame, far, 3, [255,0,0], -1)
        elif (far[0] > start[0]) & (far[0] > end[0]) & (far[1] > start[1]) & (far[1] < end[1]):
            if (angle <= 80 and d > 20):
                leftMove += 1
                cv2.circle(frame, far, 3, [0,0,0], -1)
            elif (angle >= 80 and angle <=120 and d > 30):
                leftMoveRight += 1
                cv2.circle(frame, far, 3, [100,100,100], -1)
        elif (far[1] < start[1]) & (far[1] < end[1]) & (far[0] > start[0]) & (far[0] < end[0]):
            if (angle <= 70 and d > 30):
                downMove += 1
                cv2.circle(frame, far, 3, [0,255,0], -1)
        elif (far[0] < start[0]) & (far[0] < end[0]):
            if (angle >= 70 and angle <= 110 and d > 25):
                rightMoveRight += 1
                cv2.circle(frame, far, 3, [0,0,255], -1)
            elif (angle <= 70 and d > 30):
                rightMoveAcute += 1
                cv2.circle(frame, far, 3, [255, 0, 255], -1)
        cv2.line(frame,start, end, [0,255,0], 2)
    upMove += 1
    downMove += 1
    leftMove += 1
    leftMoveRight += 1
    rightMoveAcute += 1
    rightMoveRight += 1
    return upMove, downMove, leftMove, leftMoveRight, rightMoveAcute, rightMoveRight

In [12]:
def classify(mask, frame):
    try: 
        mask = mask.astype("uint8")*255
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnt_es24 = max(contours, key = lambda x: cv2.contourArea(x))
        box_es24, boxWidth_es24, boxHeight_es24, boxAspectRatio_es24 = findBoundingBox_es24(cnt_es24)
        Xmin = box_es24[0]
        Ymin = box_es24[1]
        Xmax = box_es24[2]
        Ymax = box_es24[3]
        orientation = 'V'
        if(boxAspectRatio_es24 <= 1):
            orientation = 'H'
        leftHalfCnt = cnt_es24[(cnt_es24[:,0,0] <= box_es24[0]+boxWidth_es24/2)]
        leftHalfCntArea = cv2.contourArea(leftHalfCnt)
        rightHalfCnt = cnt_es24[(cnt_es24[:,0,0] > box_es24[0]+boxWidth_es24/2)]
        rightHalfCntArea = cv2.contourArea(rightHalfCnt)
        epsilon = 0.0005*cv2.arcLength(cnt_es24,True)
        approx= cv2.approxPolyDP(cnt_es24,epsilon,True)
        hull = cv2.convexHull(cnt_es24)
        vAnchor = np.array([(Xmax + Xmin) / 2, Ymax])
        vLines, vPoints = getVMove_es24(hull, vAnchor)
        hAnchor = np.array([Xmax, (Ymax + Ymin) / 2])
        hLines, hPoints = getHMove_es24(hull, hAnchor)
        if(orientation == 'V'):
            for i in vLines:
                cv2.line(frame, i[0], i[1], [0,255,0], 2)
            for i in vPoints:
                cv2.circle(frame, i, 3, [255,255,255], -1)
        else:
            for i in hLines:
                cv2.line(frame, i[0], i[1], [0,0,255], 2)
            for i in hPoints:
                cv2.circle(frame, i, 3, [255,255,255], -1)
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt_es24)
        arearatio = ((areahull - areacnt) / areacnt) * 100
        upMove, downMove, leftMove, leftMoveRight, rightMoveAcute, rightMoveRight = getMoveNo(approx, frame)
        font = cv2.FONT_HERSHEY_SIMPLEX
        currMove = ''
        if upMove == 1 and downMove == 1 and leftMove == 1 and leftMoveRight == 1 and rightMoveAcute == 1 and rightMoveRight == 1:
            if areacnt>2000:
                if arearatio < 8.5: 
                    if boxAspectRatio_es24 < 1.5:
                        currMove = 'normal'
                    else:
                        currMove = 'normal'
                else: 
                    if orientation == 'normal':
                        if leftHalfCntArea < rightHalfCntArea:
                            currMove = 'normal'
                        else:
                            currMove = 'normal'
                    elif orientation == 'normal' and leftHalfCntArea < rightHalfCntArea:
                        currMove = 'normal'
        elif upMove == 2 and (rightMoveRight == 2 or rightMoveAcute == 2):
            currMove = 'normal'
        elif upMove == 2:
            currMove = 'normal'
        elif (upMove == 4 and (rightMoveAcute == 2 or rightMoveRight == 2)) or (upMove == 3 and ((rightMoveAcute == 3 and rightMoveRight == 3) or rightMoveAcute == 3)):
            currMove = 'normal'
        elif upMove == 4 or (upMove == 3 and rightMoveAcute == 2) or (upMove == 2 and rightMoveAcute == 3):
            currMove = 'normal'
        elif upMove == 3:
            currMove = 'normal'
        elif rightMoveAcute == 2:
            currMove = 'un-normal' 
        elif rightMoveRight == 2:
            currMove = 'normal'   
        elif leftMove == 3:
            currMove = 'normal'
        elif leftMove == 2:
            currMove = 'normal'
        elif leftMoveRight == 2:
            currMove = 'normal'
        elif downMove == 2:
            currMove = 'normal'
        
        elif downMove == 3:
            currMove = 'normal'
        else:
            currMove = ' Reposition '
        ##################################################################
        print(currMove)
        file1 = open("myfile_video.txt", "a")
        file1.close()
        file1 = open("myfile_video.txt", "r+") 
        file1.write(currMove)
        file1.close()
        ##################################################################
        now = datetime.datetime.now()
        date_time = now.strftime("%D/%M/%Y %H:%M:%S")
        print(date_time)
        time.sleep(2)
        ##################################################################
        cv2.putText(frame,currMove,(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        cv2.imshow('frame', frame)
    except Exception as e:
        print(e)

In [ ]:
while True:
    now = datetime.datetime.now()
    date_time = now.strftime("%D/%M/%Y %H:%M:%S")
    print(date_time)
    time.sleep(2)
    try:
        mydb1 = mysql.connector.connect(host = "localhost",
                                       port = "3306",
                                       user = "root",
                                       password = "",
                                       database = "seizurs")
        MyCursor1 = mydb1.cursor()
        Sql_Query1 = "SELECT `id`, `caregiver_id` FROM `patients` WHERE `id`= 1 order by `id` desc limit 1;"
        MyCursor1.execute(Sql_Query1)
        record1 = MyCursor1.fetchall()
        mydb1.commit()
        MyCursor1.close()
        mydb1.close()
        print("MyCursor1")
        for row in record1:
            row_patients_id = row[0]
            row_patients_id = str(row_patients_id)
            print(row_patients_id)
            row_caregiver_id  = row[1]
            row_caregiver_id = str(row_caregiver_id)
            print(row_caregiver_id)
            ###################################################
            ###################################################
            print("***********************************************************************************************")
            print("***********************************************************************************************")
            print("Analysis start")
            cv2.destroyAllWindows()
            ###################################################
            ###################################################
            now = datetime.datetime.now()
            date_time = now.strftime("%D/%M/%Y %H:%M:%S")
            print(date_time)
            print("Wait New Record")
            ###################################################
            ###################################################
            if row_patients_id is not None:
                if os.path.exists("myfile_video.txt"):
                    os.remove("myfile_video.txt")
                    print("The file exist")
                else:
                    print("The file does not exist")
                ###################################################
                ################################################### 
                cap = cv2.VideoCapture(0)
                ###################################################
                ###################################################
                if not cap.isOpened():
                    raise IOError("can not open")
                else: 
                    ok_region = False
                    old_moving_region = -1 
                    _,frame1 = cap.read()
                    image3 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
                    _,frame2 = cap.read()
                    image2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                    fgbg =cv2.createBackgroundSubtractorKNN(history=1000)
                    while True:
                        ret, rgb_frame = cap.read()
                        classiferFrame = np.copy(rgb_frame)
                        classiferFrame = cv2.flip(classiferFrame,1)
                        print(classiferFrame)
                        temp_flag = np.any(classiferFrame)
                        if temp_flag == True:
                            print("***********************************************************************************************")
                            print("Analysis In Process")
                            classiferFrame = cv2.flip(classiferFrame,1)
                            rgb_frame = cv2.GaussianBlur(rgb_frame ,(7,7),cv2.BORDER_DEFAULT) 
                            image1 = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
                            rgb_frame=cv2.flip(rgb_frame,1)
                            fgmask = fgbg.apply(rgb_frame)
                            dilated_fgmask = cv2.dilate(fgmask, np.ones((3,3)), iterations = 1)
                            fgmask = np.zeros(image1.shape)
                            fgmask[dilated_fgmask>=1] = 1   
                            face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
                            gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
                            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                            for (x,y,w,h) in faces:
                                cv2.rectangle(rgb_frame,(x,y),(x+w,y+h),(255,255,0),2)
                            hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
                            ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
                            rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
                            mask = multi_color_space_model_es24(rgba_frame , hsv_frame , ycbcr_frame)
                            binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
                            binary_frame2[mask] = 1
                            for (x,y,w,h) in faces: 
                                binary_frame2[y-30:y+h+30 , x:x+w] = 0
                            eroded_frame = cv2.erode(binary_frame2, np.ones((3,3)), iterations = 1)
                            dilated_frame = cv2.dilate(eroded_frame, np.ones((10,5)), iterations = 1)
                            binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
                            temp = np.copy(binary_frame2).astype("uint8")
                            binary_frame2[dilated_frame>=1] = 1
                            temp[dilated_frame>=1] = 255
                            frame_with_hand_only = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
                            frame_with_hand_only[(binary_frame2 >=1) & (fgmask >= 1) ] = 1
                            cv2.imshow('Input', rgb_frame)
                            cv2.imshow('Final Output', frame_with_hand_only)
                            classify(frame_with_hand_only, classiferFrame)
                            image3 = image2
                            image2 = image1
                            c = cv2.waitKey(1)
                            if c == 27:
                                break
                            else:
                                print('working')
                            ###################################################
                            ###################################################
                            file=open("myfile_video.txt","r")
                            xfile = file.read()
                            xfile_response = str(xfile)
                            file.close()
                            print(xfile_response)
                            ###################################################
                            ###################################################
                            if xfile_response == 'un-normal':
                                print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
                                print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
                                mydb2 = mysql.connector.connect(host = "localhost",
                                                                port = "3306",
                                                                user = "root",
                                                                password = "",
                                                                database = "seizurs")
                                MyCursor2 = mydb2.cursor()
                                Sql_Query2 = "INSERT INTO `reports`(`patient_id`, `caregiver_id`, `datetime`, `details`, `status`) VALUES ("+row_patients_id+","+row_caregiver_id+", NOW(),'there is new un normal move', 0);"
                                MyCursor2.execute(Sql_Query2)
                                mydb2.commit()
                                MyCursor2.close()
                                mydb2.close()
                                print("MyCursor2")
                                print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
                            else:
                                print('normal')
                        else:
                            print('temp_flag false')
            else:
                print("NO RECORD IN DATABASE")
    except Exception as e:
        print(e)

05/08/24/56/2024 17:56:45
MyCursor1
1
5
***********************************************************************************************
***********************************************************************************************
Analysis start
05/08/24/56/2024 17:56:47
Wait New Record
The file exist
[[[255 253 255]
  [255 253 255]
  [255 253 255]
  ...
  [255 253 255]
  [255 253 255]
  [255 253 255]]

 [[255 253 255]
  [255 253 255]
  [255 253 255]
  ...
  [255 253 255]
  [255 253 255]
  [255 253 255]]

 [[255 253 255]
  [255 253 255]
  [255 253 255]
  ...
  [255 253 255]
  [255 253 255]
  [255 253 255]]

 ...

 [[248 226 218]
  [249 227 218]
  [250 230 219]
  ...
  [255 253 255]
  [255 253 255]
  [255 253 255]]

 [[249 226 218]
  [249 226 217]
  [249 228 217]
  ...
  [255 253 253]
  [255 253 253]
  [255 253 253]]

 [[252 228 220]
  [250 227 218]
  [249 228 217]
  ...
  [255 253 253]
  [255 253 253]
  [255 253 253]]]
******************************************************************